In [ ]:
import os
import pandas as pd
import numpy as np
import chromadb
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import display, Markdown, HTML
import re

API_KEY = "sk-FvvMLatLiWOAaqL4DsfDLg"
os.environ['LAPATHON_API_KEY'] = API_KEY

client1 = chromadb.PersistentClient(path="./chroma_db")
client2 = chromadb.PersistentClient(path="./chroma_db_storage")
collection1 = client1.get_or_create_collection(name="textbook_topics_index")
collection2 = client2.get_or_create_collection(name="ukrainian_language_rag")

if collection1.count() > 0:
    client, collection = client1, collection1
elif collection2.count() > 0:
    client, collection = client2, collection2
else:
    client, collection = client1, collection1

def identify_topic(question, n_results=1, use_api=True, use_parquet_embeddings=True, grade=None, subject=None):
    where_filter = {}
    if grade is not None:
        where_filter['grade'] = str(grade)
    if subject is not None:
        where_filter['global_discipline_name'] = subject
    
    if use_api:
        print("Using API")
        try:
            api_key = os.environ.get('LAPATHON_API_KEY', '')
            if not api_key or api_key == "YOUR_TEAM_API_KEY":
                raise ValueError("LAPATHON_API_KEY не встановлено")
            
            client_openai = OpenAI(api_key=api_key, base_url="http://146.59.127.106:4000")
            response = client_openai.embeddings.create(
                input=question, model="text-embedding-qwen", encoding_format="float"
            )
            query_embedding = response.data[0].embedding
            query_kwargs = {'query_embeddings': [list(query_embedding)], 'n_results': n_results}
            if where_filter:
                query_kwargs['where'] = where_filter
            results = collection.query(**query_kwargs)
        except Exception:
            use_api = False
            if use_parquet_embeddings:
                try:
                    df_toc = pd.read_parquet(r'src\dataset\qwen_emb\text-embedding-qwen\toc_for_hackathon_with_subtopics.parquet')
                    df_toc = df_toc.dropna(subset=['book_topic_id']).copy()
                    if grade is not None:
                        df_toc = df_toc[df_toc['grade'] == grade].copy()
                    if subject is not None:
                        df_toc = df_toc[df_toc['global_discipline_name'] == subject].copy()
                    topic_texts_list = df_toc['topic_text'].values.tolist()
                    vectorizer = TfidfVectorizer(max_features=1000, stop_words=None)
                    all_texts = topic_texts_list + [question]
                    tfidf_matrix = vectorizer.fit_transform(all_texts)
                    query_tfidf = tfidf_matrix[-1:].toarray()
                    topics_tfidf = tfidf_matrix[:-1].toarray()
                    similarities = cosine_similarity(query_tfidf, topics_tfidf)[0]
                    best_match_idx = np.argmax(similarities)
                    best_match_embedding = list(df_toc.iloc[best_match_idx]['topic_embedding'])
                    query_kwargs = {'query_embeddings': [best_match_embedding], 'n_results': n_results}
                    if where_filter:
                        query_kwargs['where'] = where_filter
                    results = collection.query(**query_kwargs)
                except Exception:
                    use_api = False
    
    if not use_api:
        print("Using Parquet")
        df_toc = pd.read_parquet(r'src\dataset\qwen_emb\text-embedding-qwen\toc_for_hackathon_with_subtopics.parquet')
        df_toc = df_toc.dropna(subset=['book_topic_id']).copy()
        if grade is not None:
            df_toc = df_toc[df_toc['grade'] == grade].copy()
        if subject is not None:
            df_toc = df_toc[df_toc['global_discipline_name'] == subject].copy()
        topic_texts_list = df_toc['topic_text'].values.tolist()
        topic_metadata_list = df_toc[['book_name', 'grade', 'section_title', 'topic_title', 'book_topic_id',
                                      'topic_start_page', 'topic_end_page', 'book_id', 'topic_type']].to_dict('records')
        vectorizer = TfidfVectorizer(max_features=1000, stop_words=None)
        all_texts = topic_texts_list + [question]
        tfidf_matrix = vectorizer.fit_transform(all_texts)
        query_embedding = tfidf_matrix[-1:].toarray()
        topic_embeddings_tfidf = tfidf_matrix[:-1].toarray()
        similarities = cosine_similarity(query_embedding, topic_embeddings_tfidf)[0]
        top_indices = np.argsort(similarities)[::-1][:n_results]
        results = {
            'ids': [[df_toc.iloc[idx]['book_topic_id'] for idx in top_indices]],
            'metadatas': [[topic_metadata_list[idx] for idx in top_indices]],
            'documents': [[topic_texts_list[idx] for idx in top_indices]],
            'distances': [[1 - float(similarities[idx]) for idx in top_indices]]
        }
    
    formatted_results = []
    if results['ids'] and len(results['ids'][0]) > 0:
        for i in range(len(results['ids'][0])):
            metadata = results['metadatas'][0][i].copy()
            if 'topic_start_page' not in metadata or 'topic_end_page' not in metadata:
                df_toc_full = pd.read_parquet(r'src\dataset\qwen_emb\text-embedding-qwen\toc_for_hackathon_with_subtopics.parquet')
                topic_id = str(metadata.get('book_topic_id', ''))
                if topic_id:
                    topic_row = df_toc_full[df_toc_full['book_topic_id'] == topic_id]
                    if not topic_row.empty:
                        if 'topic_start_page' not in metadata:
                            metadata['topic_start_page'] = topic_row.iloc[0].get('topic_start_page')
                        if 'topic_end_page' not in metadata:
                            metadata['topic_end_page'] = topic_row.iloc[0].get('topic_end_page')
                        if 'book_id' not in metadata:
                            metadata['book_id'] = topic_row.iloc[0].get('book_id')
            formatted_results.append({
                'metadata': metadata,
                'document_text': results['documents'][0][i],
                'similarity': 1 - results['distances'][0][i] if 'distances' in results and results['distances'][0] else None
            })
    return formatted_results

def get_pages_for_topic(topic_metadata, max_pages=None):
    df_pages = pd.read_parquet(r'src\dataset\qwen_emb\text-embedding-qwen\pages_for_hackathon.parquet')
    book_topic_id = str(topic_metadata.get('book_topic_id', ''))
    if not book_topic_id:
        return []
    topic_pages = df_pages[df_pages['book_topic_id'] == book_topic_id].copy()
    if 'topic_start_page' in topic_metadata and 'topic_end_page' in topic_metadata:
        start_page = topic_metadata.get('topic_start_page')
        end_page = topic_metadata.get('topic_end_page')
        if pd.notna(start_page) and pd.notna(end_page):
            topic_pages = topic_pages[(topic_pages['book_page_number'] >= int(start_page)) & 
                                     (topic_pages['book_page_number'] <= int(end_page))]
    topic_pages = topic_pages.sort_values('book_page_number')
    if max_pages is not None and len(topic_pages) > max_pages:
        topic_pages = topic_pages.head(max_pages)
    return [{
        'page_number': int(row['book_page_number']),
        'page_text': str(row['page_text']),
        'section_title': str(row.get('section_title', '')),
        'topic_title': str(row.get('topic_title', '')),
        'page_filename': str(row.get('page_filename', ''))
    } for _, row in topic_pages.iterrows()]

def get_context_for_llm(topic_result, max_pages=10):
    metadata = topic_result['metadata']
    topic_text = topic_result.get('document_text', '')
    pages = get_pages_for_topic(metadata, max_pages=max_pages)
    context_parts = [
        f"# {metadata.get('topic_title', 'Тема')}",
        f"**Розділ:** {metadata.get('section_title', '')}",
        f"**Клас:** {metadata.get('grade', '')}",
        ""
    ]
    if topic_text:
        context_parts.extend(["## Зміст теми", topic_text, ""])
    if pages:
        context_parts.append(f"## Текст сторінок ({len(pages)} сторінок)")
        for page in pages:
            context_parts.extend([f"\n### Сторінка {page['page_number']}", page['page_text'], ""])
    else:
        context_parts.extend(["## Текст сторінок", "*Сторінки не знайдено*"])
    return "\n".join(context_parts)

def identify_relevant_pages(question, n_results=5, use_api=True, grade=None, subject=None):
    if use_api:
        try:
            api_key = os.environ.get('LAPATHON_API_KEY', '')
            if not api_key or api_key == "YOUR_TEAM_API_KEY":
                raise ValueError("LAPATHON_API_KEY не встановлено")
            
            client_openai = OpenAI(api_key=api_key, base_url="http://146.59.127.106:4000")
            response = client_openai.embeddings.create(
                input=question, model="text-embedding-qwen", encoding_format="float"
            )
            query_embedding = response.data[0].embedding
            
            df_pages = pd.read_parquet(r'src\dataset\qwen_emb\text-embedding-qwen\pages_for_hackathon.parquet')
            df_pages = df_pages.dropna(subset=['page_text_embedding']).copy()
            if grade is not None:
                df_pages = df_pages[df_pages['grade'] == grade].copy()
            if subject is not None:
                df_pages = df_pages[df_pages['global_discipline_name'] == subject].copy()
            
            page_embeddings = np.array([list(emb) for emb in df_pages['page_text_embedding'].values])
            similarities = cosine_similarity([query_embedding], page_embeddings)[0]
            top_indices = np.argsort(similarities)[::-1][:n_results]
            
            results = []
            for idx in top_indices:
                row = df_pages.iloc[idx]
                results.append({
                    'page_text': str(row['page_text']),
                    'page_number': int(row['book_page_number']),
                    'similarity': float(similarities[idx]),
                    'metadata': {
                        'book_name': str(row.get('book_name', '')),
                        'grade': str(row.get('grade', '')),
                        'section_title': str(row.get('section_title', '')),
                        'topic_title': str(row.get('topic_title', '')),
                        'book_topic_id': str(row.get('book_topic_id', '')),
                        'book_id': str(row.get('book_id', '')),
                        'global_discipline_name': str(row.get('global_discipline_name', ''))
                    }
                })
            return results
        except Exception as e:
            use_api = False
    
    if not use_api:
        df_pages = pd.read_parquet(r'src\dataset\qwen_emb\text-embedding-qwen\pages_for_hackathon.parquet')
        df_pages = df_pages.dropna(subset=['page_text']).copy()
        if grade is not None:
            df_pages = df_pages[df_pages['grade'] == grade].copy()
        if subject is not None:
            df_pages = df_pages[df_pages['global_discipline_name'] == subject].copy()
        
        page_texts_list = df_pages['page_text'].values.tolist()
        vectorizer = TfidfVectorizer(max_features=1000, stop_words=None)
        all_texts = page_texts_list + [question]
        tfidf_matrix = vectorizer.fit_transform(all_texts)
        query_tfidf = tfidf_matrix[-1:].toarray()
        pages_tfidf = tfidf_matrix[:-1].toarray()
        similarities = cosine_similarity(query_tfidf, pages_tfidf)[0]
        top_indices = np.argsort(similarities)[::-1][:n_results]
        
        results = []
        for idx in top_indices:
            row = df_pages.iloc[idx]
            results.append({
                'page_text': str(row['page_text']),
                'page_number': int(row['book_page_number']),
                'similarity': float(similarities[idx]),
                'metadata': {
                    'book_name': str(row.get('book_name', '')),
                    'grade': str(row.get('grade', '')),
                    'section_title': str(row.get('section_title', '')),
                    'topic_title': str(row.get('topic_title', '')),
                    'book_topic_id': str(row.get('book_topic_id', '')),
                    'book_id': str(row.get('book_id', '')),
                    'global_discipline_name': str(row.get('global_discipline_name', ''))
                }
            })
        return results

def get_context_from_pages(page_results, max_length=5000):
    context_parts = []
    total_length = 0
    for i, page in enumerate(page_results, 1):
        if total_length >= max_length:
            break
        page_text = page['page_text']
        remaining = max_length - total_length
        if len(page_text) > remaining:
            page_text = page_text[:remaining] + "..."
        
        context_parts.append(f"### Сторінка {page['page_number']} (схожість: {page['similarity']:.4f})")
        context_parts.append(f"**Тема:** {page['metadata']['topic_title']}")
        context_parts.append(f"**Розділ:** {page['metadata']['section_title']}")
        context_parts.append("")
        context_parts.append(page_text)
        context_parts.append("")
        
        total_length += len(page_text)
    
    return "\n".join(context_parts)

def full_rag_workflow(question, n_topics=1, max_pages=5, use_llm=False, grade=None, subject=None):
    results = identify_topic(question, n_results=n_topics, use_api=True, use_parquet_embeddings=True, grade=grade, subject=subject)
    if not results:
        return None
    best_topic = results[0]
    pages = get_pages_for_topic(best_topic['metadata'], max_pages=max_pages)
    context = get_context_for_llm(best_topic, max_pages=max_pages)
    llm_response = None
    if use_llm:
        try:
            api_key = os.environ.get('LAPATHON_API_KEY', '')
            client_openai = OpenAI(api_key=api_key, base_url="http://146.59.127.106:4000")
            prompt = f"""Ти - вчитель математики. Користувач задав питання про геометричну прогресію.

КОНТЕКСТ З ПІДРУЧНИКА:
{context[:3000]}

ПИТАННЯ КОРИСТУВАЧА:
{question}

Дай детальну відповідь на основі наданого контексту з підручника. Поясни крок за кроком."""
            response = client_openai.chat.completions.create(
                model="lapa",
                messages=[
                    {"role": "system", "content": "Ти - вчитель математики, який допомагає учням зрозуміти матеріал."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=1000
            )
            llm_response = response.choices[0].message.content
        except Exception:
            pass
    return {'question': question, 'topic': best_topic, 'pages': pages, 'context': context, 'llm_response': llm_response}

question = "Чому дорівнює знаменник геометричної прогресії (b_n), якщо b_1=56, b_2=7?"

grade_filter = 9
subject_filter = "Алгебра"

print("Доступні значення:")
print("  Класи: 8, 9")
print("  Предмети: 'Алгебра', 'Українська мова', 'Історія України'")
print("  (Можна встановити grade=None або subject=None для пошуку без фільтрів)\n")

print("=== Пошук по темах ===")
print(f"Фільтри: Клас={grade_filter}, Предмет={subject_filter}")
result = identify_topic(question, n_results=1, grade=grade_filter, subject=subject_filter)
if result:
    print(f"Тема: {result[0]['metadata']['topic_title']}")
    print(f"Розділ: {result[0]['metadata'].get('section_title', '')}")
    print(f"Схожість: {result[0]['similarity']:.4f}" if result[0]['similarity'] else "")

print("\n=== Пошук по тексту сторінок ===")
print(f"Фільтри: Клас={grade_filter}, Предмет={subject_filter}")
pages_result = identify_relevant_pages(question, n_results=3, grade=grade_filter, subject=subject_filter)
if pages_result:
    for i, page in enumerate(pages_result, 1):
        print(f"\n{'='*80}")
        print(f"{i}. Сторінка {page['page_number']} (схожість: {page['similarity']:.4f})")
        print(f"Розділ: {page['metadata']['section_title']}")
        print(f"Тема: {page['metadata']['topic_title']}")
        print(f"Клас: {page['metadata'].get('grade', '')}")
        print(f"\nПовний текст сторінки:")
        print("-"*80)
        display(Markdown(page['page_text']))
        print("-"*80)
    
    context = get_context_from_pages(pages_result, max_length=3000)
    print(f"\nКонтекст зі сторінок: {len(context)} символів")

Доступні значення:
  Класи: 8, 9
  Предмети: 'Алгебра', 'Українська мова', 'Історія України'
  (Можна встановити grade=None або subject=None для пошуку без фільтрів)

=== Пошук по темах ===
Фільтри: Клас=9, Предмет=Алгебра
Using API
Using Parquet
Тема: § 20. Сума n перших членів геометричної прогресії
Розділ: Розділ 3. Числові послідовності
Схожість: 0.4907

=== Пошук по тексту сторінок ===
Фільтри: Клас=9, Предмет=Алгебра

1. Сторінка 170 (схожість: 0.8192)
Розділ: Розділ 3. Числові послідовності
Тема: § 18. Геометрична прогресія, її властивості. Формула n-го члена геометричної прогресії
Клас: 9

Повний текст сторінки:
--------------------------------------------------------------------------------


РОЗДІЛ 3
$\frac{\cdot 2}{3}$; $\frac{\cdot 2}{6}$; $\frac{\cdot 2}{12}$; $\frac{\cdot 2}{24}$; $\frac{\cdot 2}{48}$; $\frac{\cdot 2}{96}$; ...
Таку послідовність називають геометричною прогресією.
**Геометричною прогресією називають послідовність відмінних від нуля чисел, кожне з яких, починаючи з другого, дорівнює попередньому, помноженому на одне й те саме число.**
Це число називають знаменником геометричної прогресії і позначають буквою $q$ (від першої літери французького слова *quotient* – частка). Тому якщо ($b_n$) – геометрична прогресія, справджуються рівності:
$b_2 = b_1q$; $b_3 = b_2q$; $b_4 = b_3q$; ... .
Отже, для будь-якого натурального $n$ матимемо:
$b_{n+1} = b_nq$.
Тоді
$q = \frac{b_{n+1}}{b_n}$,
тобто
**знаменник геометричної прогресії можна знайти, якщо будь-який член прогресії, починаючи з другого, поділити на попередній.**
Зауважимо, що оскільки члени геометричної прогресії відмінні від нуля, то і знаменник $q$ не може дорівнювати нулю, тобто $q \ne 0$.
Якщо $q = 1$, то геометрична прогресія складатиметься з однакових чисел. Наприклад, якщо $b_1 = -5$ і $q = 1$, то матимемо геометричну прогресію:
-5; -5; -5; -5; -5; ... .
Зауважимо, що отриману послідовність можна також вважати і арифметичною прогресією, перший член якої дорівнює -5, а різниця дорівнює 0.
Нехай перший член геометричної прогресії дорівнює $b_1$, а знаменник дорівнює $q$. Тоді
$b_2 = b_1q$;
$b_3 = b_2q = (b_1q)q = b_1q^2$;
$b_4 = b_3q = (b_1q^2)q = b_1q^3$;
$b_5 = b_4q = (b_1q^3)q = b_1q^4$ і т. д.
170
Право для безоплатного розміщення підручника в мережі Інтернет має
Міністерство освіти і науки України http://mon.gov.ua/ та Інститут модернізації змісту освіти https://imzo.gov.ua

--------------------------------------------------------------------------------

2. Сторінка 176 (схожість: 0.8190)
Розділ: Розділ 3. Числові послідовності
Тема: § 18. Геометрична прогресія, її властивості. Формула n-го члена геометричної прогресії
Клас: 9

Повний текст сторінки:
--------------------------------------------------------------------------------


РОЗДІЛ 3
780. Знайдіть знаменник геометричної прогресії ($b_n$), у якої:
1) $b_{10} = 11, b_{12} = 99$;
2) $b_{10} = 27, b_{13} = 1$.
781. Запишіть геометричну прогресію з п’яти членів, у якої
третій член дорівнює 10, а знаменник дорівнює –2.
782. Запишіть геометричну прогресію із шести членів, у якої
четвертий член дорівнює 80, а знаменник дорівнює –4.
783. Послідовність ($x_n$) – геометрична прогресія. Знайдіть $x_5$,
якщо $x_1 = \frac{1}{2}, x_3 = \frac{1}{8}$.
784. Послідовність ($b_n$) – геометрична прогресія. Знайдіть $b_1$,
якщо $b_4 = -1, b_6 = -100$.
785. Послідовність ($c_n$) – геометрична прогресія. Знайдіть:
1) $c_1$, якщо $c_3 = 10, c_5 = \frac{1}{10}$;
2) $c_6$, якщо $c_1 = 2, c_3 = 8$.
786. Під мікроскопом розглядають 5 клітин, які розмно-
жуються поділом навпіл щохвилини. Скільки утвориться
клітин через одну хвилину; через три хвилини; через шість
хвилин?

4 Високий рівень
787. Між числами 1 і 64 вставте: 1) одне число; 2) два числа
таких, щоб вони разом з даними утворили геометричну
прогресію.
788. Геометрична прогресія ($b_n$) складається з п’яти членів:
$\frac{1}{2}, x_2, x_3, 4, x_5$. Знайдіть $x_2, x_3, x_5$.
789. При якому значенні $x$ числа $x + 3, 2x$ і $5x – 4$ є послі-
довними членами геометричної прогресії? Знайдіть ці числа.
790. При якому значенні $y$ числа $y, 2y + 3$ і $4y + 3$ є послі-
довними членами геометричної прогресії? Знайдіть ці числа.
791. Доведіть, що коли числа $a, b$ і $c$ є послідовними членами
геометричної прогресії, то справджується рівність:
$(a + b + c)(a – b + c) = a^2 + b^2 + c^2$.

176
Право для безоплатного розміщення підручника в мережі Інтернет має
Міністерство освіти і науки України http://mon.gov.ua/ та Інститут модернізації змісту освіти https://imzo.gov.ua

--------------------------------------------------------------------------------

3. Сторінка 175 (схожість: 0.8153)
Розділ: Розділ 3. Числові послідовності
Тема: § 18. Геометрична прогресія, її властивості. Формула n-го члена геометричної прогресії
Клас: 9

Повний текст сторінки:
--------------------------------------------------------------------------------


Числові послідовності

**769.** Послідовність ($b_n$) – геометрична прогресія. Знайдіть:
1) $b_6$, якщо $b_1 = 1, q = 2$;
2) $b_5$, якщо $b_1 = 125, q = -\frac{1}{5}$;
3) $b_7$, якщо $b_1 = 64, q = \frac{1}{2}$;
4) $b_4$, якщо $b_1 = \sqrt{2}, q = \frac{1}{\sqrt{2}}$.

**770.** Послідовність ($b_n$) – геометрична прогресія. Знайдіть:
1) $b_5$, якщо $b_1 = 2, q = -1$;
2) $b_4$, якщо $b_1 = -128, q = \frac{1}{2}$;
3) $b_7$, якщо $b_1 = 64, q = -\frac{1}{4}$;
4) $b_3$, якщо $b_1 = 3, q = -\sqrt{2}$.

**771.** Знайдіть шостий та $n$-й члени геометричної прогресії:
1) 10 000; 1000; 100;
2) 3; -6; 12;

**772.** Знайдіть п’ятий та $n$-й члени геометричної прогресії:
1) 20; 5; 1,25;
2) 4; -8; 16;

**773.** Знайдіть перший член геометричної прогресії ($b_n$), якщо:
1) $b_4 = 40, q = 2$;
2) $b_3 = 27, q = -3$.

**774.** Знайдіть перший член геометричної прогресії ($b_n$), якщо:
1) $b_3 = 100, q = -2$;
2) $b_4 = 64, q = 4$.

**775.** Послідовність ($b_n$) – геометрична прогресія. Знайдіть $b_7$, якщо $b_6 = 4, b_8 = 9$.

**776.** Матеріальна точка за першу секунду подолала 5 м, а за кожну наступну – утричі більшу за попередню відстань. Скільки метрів подолала матеріальна точка за четверту секунду?

**777.** Ламана складається з п’яти ланок. Перша з них дорівнює 48 см, а кожна наступна вдвічі коротша за попередню. Яка довжина п’ятої (найкоротшої) ланки?

**3 Достатній рівень**

**778.** Доведіть, що послідовність ($x_n$), задана формулою $x_n = 3 \cdot 4^n$, є геометричною прогресією. Знайдіть її перший член і знаменник.

**779.** Знайдіть знаменник геометричної прогресії ($b_n$), у якої:
1) $b_7 = 12, b_9 = 48$;
2) $b_8 = 9, b_{11} = 243$;
3) $b_{16} = 16, b_{18} = 9$;
4) $b_{30} = 1, b_{33} = -1$.

Право для безоплатного розміщення підручника в мережі Інтернет має
Міністерство освіти і науки України http://mon.gov.ua/ та Інститут модернізації змісту освіти https://imzo.gov.ua
175

--------------------------------------------------------------------------------

Контекст зі сторінок: 3362 символів


df_question = pd.read

In [2]:
df_question = pd.read_parquet(r'src\dataset\lms_questions_dev.parquet')




In [3]:
df_question.columns

Index(['question_id', 'question_text', 'test_type', 'description', 'model',
       'source', 'global_discipline_name', 'grade', 'answers',
       'correct_answer_indices'],
      dtype='object')

In [4]:
import re
def format_latex_for_markdown(text):
    """Конвертує LaTeX синтаксис \( \) в Markdown синтаксис $ для правильного відображення."""
    if not text:
        return text
    text = str(text)
    text = re.sub(r'\\\(', '$', text)
    text = re.sub(r'\\\)', '$', text)
    text = re.sub(r'\\\[', '$$', text)
    text = re.sub(r'\\\]', '$$', text)
    return text

def solve_question_with_rag(question_text, answers, grade, subject, use_pages=True, n_results=5, max_context_length=4000):
    """Отримує контекст через RAG та викликає LLM для відповіді на питання."""
    try:
        if use_pages:
            page_results = identify_relevant_pages(question_text, n_results=n_results, grade=grade, subject=subject)
            if not page_results:
                return None, "Не знайдено релевантних сторінок"
            context = get_context_from_pages(page_results, max_length=max_context_length)
        else:
            topic_results = identify_topic(question_text, n_results=1, grade=grade, subject=subject)
            if not topic_results:
                return None, "Не знайдено релевантних тем"
            context = get_context_for_llm(topic_results[0], max_pages=5)
        
        answers_text = "\n".join([f"{chr(65+i)}. {ans}" for i, ans in enumerate(answers)])
        
        prompt = f"""Ти - вчитель, який допомагає учням відповідати на тестові питання.

КОНТЕКСТ З ПІДРУЧНИКА:
{context}

ПИТАННЯ:
{question_text}

ВАРІАНТИ ВІДПОВІДЕЙ:
{answers_text}

Проаналізуй питання на основі наданого контексту з підручника. Поясни своє міркування та обґрунтуй, чому обрана відповідь правильна. В кінці вкажи літеру правильної відповіді (A, B, C або D)."""
        
        api_key = os.environ.get('LAPATHON_API_KEY', '')
        client_openai = OpenAI(api_key=api_key, base_url="http://146.59.127.106:4000")
        
        response = client_openai.chat.completions.create(
            model="lapa",
            messages=[
                {"role": "system", "content": "Ти - вчитель, який точно відповідає на тестові питання на основі наданого контексту. Ти завжди пояснюєш своє міркування."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )
        
        llm_full_response = response.choices[0].message.content.strip()
        llm_answer = llm_full_response.upper()
        return llm_full_response, context
    except Exception as e:
        return None, f"Помилка: {str(e)}"

def extract_answer_index(llm_answer):
    """Витягує індекс відповіді з тексту LLM (A=0, B=1, C=2, D=3)."""
    if not llm_answer:
        return None
    llm_answer = llm_answer.strip().upper()
    for i, letter in enumerate(['A', 'B', 'C', 'D']):
        if letter in llm_answer:
            return i
    return None

def run_benchmark(df_questions, max_questions=None, use_pages=True, n_results=5):
    """Запускає бенчмарк на датасеті питань."""
    results = []
    
    questions_to_process = df_questions.head(max_questions) if max_questions else df_questions
    
    for idx, row in questions_to_process.iterrows():
        question_id = row['question_id']
        question_text = row['question_text']
        answers = row['answers']
        correct_indices = row['correct_answer_indices']
        grade = row['grade']
        subject = row['global_discipline_name']
        
        print(f"\n[{idx+1}/{len(questions_to_process)}] Обробка питання {question_id}")
        print(f"Предмет: {subject}, Клас: {grade}")
        print(f"Питання:")
        display(Markdown(format_latex_for_markdown(question_text)))
        
        llm_full_response, context = solve_question_with_rag(
            question_text, answers, grade, subject, 
            use_pages=use_pages, n_results=n_results
        )
        
        predicted_index = extract_answer_index(llm_full_response)
        is_correct = predicted_index in correct_indices if predicted_index is not None else False
        
        result = {
            'question_id': question_id,
            'question_text': question_text,
            'grade': grade,
            'subject': subject,
            'correct_indices': correct_indices,
            'predicted_index': predicted_index,
            'llm_answer_raw': llm_full_response,
            'is_correct': is_correct,
            'context_length': len(context) if context else 0
        }
        results.append(result)
        
        status = "✓" if is_correct else "✗"
        correct_letters = [chr(65+i) for i in correct_indices]
        predicted_letter = chr(65+predicted_index) if predicted_index is not None else 'N/A'
        
        print(f"{status} Правильна відповідь: {correct_letters}, LLM відповідь: {predicted_letter}")
        print(f"\nМіркування LLM:")
        print("-" * 80)
        display(Markdown(format_latex_for_markdown(llm_full_response)))
        print("-" * 80)
    
    return pd.DataFrame(results)

def calculate_metrics(results_df):
    """Обчислює метрики бенчмарку."""
    total = len(results_df)
    correct = results_df['is_correct'].sum()
    accuracy = correct / total if total > 0 else 0
    
    metrics = {
        'total_questions': total,
        'correct_answers': correct,
        'accuracy': accuracy,
        'accuracy_percent': accuracy * 100
    }
    
    if 'subject' in results_df.columns:
        subject_metrics = results_df.groupby('subject').agg({
            'is_correct': ['count', 'sum']
        }).reset_index()
        subject_metrics.columns = ['subject', 'count', 'correct']
        subject_metrics['accuracy'] = subject_metrics['correct'] / subject_metrics['count']
        metrics['by_subject'] = subject_metrics
    
    if 'grade' in results_df.columns:
        grade_metrics = results_df.groupby('grade').agg({
            'is_correct': ['count', 'sum']
        }).reset_index()
        grade_metrics.columns = ['grade', 'count', 'correct']
        grade_metrics['accuracy'] = grade_metrics['correct'] / grade_metrics['count']
        metrics['by_grade'] = grade_metrics
    
    return metrics

print("Бенчмарк функції завантажено!")
print("Використання:")
print("  results_df = run_benchmark(df_question, max_questions=10, use_pages=True)")
print("  metrics = calculate_metrics(results_df)")
print("  print(metrics)")

Бенчмарк функції завантажено!
Використання:
  results_df = run_benchmark(df_question, max_questions=10, use_pages=True)
  metrics = calculate_metrics(results_df)
  print(metrics)


<>:3: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\('
C:\Users\vladm\AppData\Local\Temp\ipykernel_9460\1750407156.py:3: SyntaxWarning: invalid escape sequence '\('
  """Конвертує LaTeX синтаксис \( \) в Markdown синтаксис $ для правильного відображення."""


In [5]:
# Запуск бенчмарку на всіх питаннях
print("Запуск бенчмарку на всіх питаннях з датасету...")
print(f"Всього питань: {len(df_question)}")
print("="*80)

results_df = run_benchmark(df_question, max_questions=None, use_pages=True, n_results=5)

print("\n" + "="*80)
print("РЕЗУЛЬТАТИ БЕНЧМАРКУ")
print("="*80)

metrics = calculate_metrics(results_df)
print(f"\nЗагальна точність: {metrics['accuracy_percent']:.2f}% ({metrics['correct_answers']}/{metrics['total_questions']})")

print("\n" + "="*80)
print("СТАТИСТИКА ПО ПРЕДМЕТАХ")
print("="*80)

if 'by_subject' in metrics:
    subject_stats = metrics['by_subject'].copy()
    subject_stats['accuracy_percent'] = subject_stats['accuracy'] * 100
    subject_stats = subject_stats.sort_values('accuracy_percent', ascending=False)
    
    print("\nПредмет | Кількість питань | Правильних | Точність (%)")
    print("-" * 60)
    for _, row in subject_stats.iterrows():
        subject = row['subject']
        count = int(row['count'])
        correct = int(row['correct'])
        accuracy_pct = row['accuracy_percent']
        print(f"{subject:25} | {count:15} | {correct:10} | {accuracy_pct:11.2f}%")
    
    print("\nДетальна таблиця:")
    print(subject_stats[['subject', 'count', 'correct', 'accuracy_percent']].to_string(index=False))
else:
    if 'subject' in results_df.columns:
        subject_stats = results_df.groupby('subject').agg({
            'is_correct': ['count', 'sum']
        }).reset_index()
        subject_stats.columns = ['subject', 'count', 'correct']
        subject_stats['accuracy'] = subject_stats['correct'] / subject_stats['count']
        subject_stats['accuracy_percent'] = subject_stats['accuracy'] * 100
        subject_stats = subject_stats.sort_values('accuracy_percent', ascending=False)
        
        print("\nПредмет | Кількість питань | Правильних | Точність (%)")
        print("-" * 60)
        for _, row in subject_stats.iterrows():
            subject = row['subject']
            count = int(row['count'])
            correct = int(row['correct'])
            accuracy_pct = row['accuracy_percent']
            print(f"{subject:25} | {count:15} | {correct:10} | {accuracy_pct:11.2f}%")
        
        print("\nДетальна таблиця:")
        print(subject_stats[['subject', 'count', 'correct', 'accuracy_percent']].to_string(index=False))

print("\n" + "="*80)
print("СТАТИСТИКА ПО КЛАСАХ")
print("="*80)

if 'by_grade' in metrics:
    grade_stats = metrics['by_grade'].copy()
    grade_stats['accuracy_percent'] = grade_stats['accuracy'] * 100
    grade_stats = grade_stats.sort_values('grade')
    
    print("\nКлас | Кількість питань | Правильних | Точність (%)")
    print("-" * 50)
    for _, row in grade_stats.iterrows():
        grade = int(row['grade'])
        count = int(row['count'])
        correct = int(row['correct'])
        accuracy_pct = row['accuracy_percent']
        print(f"{grade:4} | {count:15} | {correct:10} | {accuracy_pct:11.2f}%")
    
    print("\nДетальна таблиця:")
    print(grade_stats[['grade', 'count', 'correct', 'accuracy_percent']].to_string(index=False))

print("\nДетальні результати:")
print(results_df[['question_id', 'subject', 'grade', 'is_correct', 'predicted_index']])

print("\n" + "="*80)
print("ПРИКЛАДИ МІРКУВАНЬ LLM")
print("="*80)

for idx, row in results_df.head(3).iterrows():
    print(f"\n{'='*80}")
    print(f"Питання {idx+1}")
    print(f"{'='*80}")
    print(f"Предмет: {row['subject']}, Клас: {row['grade']}")
    print(f"Правильна відповідь: {[chr(65+i) for i in row['correct_indices']]}")
    print(f"LLM відповідь: {chr(65+row['predicted_index']) if row['predicted_index'] is not None else 'N/A'}")
    print(f"Статус: {'✓ Правильно' if row['is_correct'] else '✗ Неправильно'}")
    print(f"\nТекст питання:")
    display(Markdown(format_latex_for_markdown(row['question_text'])))
    print(f"\nМіркування LLM:")
    print("-" * 80)
    display(Markdown(format_latex_for_markdown(row['llm_answer_raw'])))
    print("-" * 80)

print("\n" + "="*80)
print("ЗБЕРЕЖЕННЯ РЕЗУЛЬТАТІВ")
print("="*80)
results_df.to_csv('benchmark_results.csv', index=False, encoding='utf-8')
print("Результати збережено в benchmark_results.csv")

Запуск бенчмарку на всіх питаннях з датасету...
Всього питань: 141

[1/141] Обробка питання 4219882a-11a5-4d12-a319-aef44197bbea
Предмет: Алгебра, Клас: 9
Питання:


Чому дорівнює знаменник геометричної прогресії $(b_n)$, якщо $ b_1=56,  b_2=7$?

✗ Правильна відповідь: ['D'], LLM відповідь: A

Міркування LLM:
--------------------------------------------------------------------------------


Щоб знайти знаменник геометричної прогресії, ми можемо використати формулу:

$$ q = \frac{b_{n+1}}{b_n} $$

У цьому випадку нам дано $ b_1 = 56 $ та $ b_2 = 7 $. Ми можемо підставити ці значення у формулу, щоб знайти знаменник:

$$ q = \frac{7}{56} $$

Спрощуючи дріб, отримуємо:

$$ q = \frac{1}{8} $$

Отже, знаменник геометричної прогресії дорівнює $\frac{1}{8}$. Правильна відповідь – D.

--------------------------------------------------------------------------------

[2/141] Обробка питання ab6163d7-3b31-41f9-9505-c7b94877bc9b
Предмет: Історія України, Клас: 9
Питання:


...

Щоб відповісти на це питання, нам потрібно визначити, яке з наведених речень є неповним і містить тире.

1. Життя – це боротьба.
2. Боротись – значить жить.
3. Життя триває довго, щастя – мить.
4. І чебрець, і м’яту, і любисток – усе вирощувала мати.

Речення 1 і 3 є повними, оскільки містять усі необхідні члени речення. Речення 2 є неповним, але не містить тире. Речення 4 є повним, оскільки містить усі необхідні члени речення.

Отже, правильна відповідь – B. Боротись – значить жить.

--------------------------------------------------------------------------------

[10/141] Обробка питання b709f8aa-ec23-43d4-b4c6-c77c06892ca0
Предмет: Українська мова, Клас: 8
Питання:


Безособове речення, у якому головним членом є особове дієслово в значенні безособового

C:\Users\vladm\AppData\Local\Temp\ipykernel_9460\1750407156.py:3: SyntaxWarning: invalid escape sequence '\('
  """Конвертує LaTeX синтаксис \( \) в Markdown синтаксис $ для правильного відображення."""


KeyboardInterrupt: 

In [ ]:
# Обчислення статистики з існуючого results_df
print("="*80)
print("СТАТИСТИКА ПО ПРЕДМЕТАХ")
print("="*80)

if 'subject' in results_df.columns:
    subject_stats = results_df.groupby('subject').agg({
        'is_correct': ['count', 'sum']
    }).reset_index()
    subject_stats.columns = ['subject', 'count', 'correct']
    subject_stats['accuracy'] = subject_stats['correct'] / subject_stats['count']
    subject_stats['accuracy_percent'] = subject_stats['accuracy'] * 100
    subject_stats = subject_stats.sort_values('accuracy_percent', ascending=False)
    
    print("\nПредмет | Кількість питань | Правильних | Точність (%)")
    print("-" * 60)
    for _, row in subject_stats.iterrows():
        subject = row['subject']
        count = int(row['count'])
        correct = int(row['correct'])
        accuracy_pct = row['accuracy_percent']
        print(f"{subject:25} | {count:15} | {correct:10} | {accuracy_pct:11.2f}%")
    
    print("\nДетальна таблиця:")
    print(subject_stats[['subject', 'count', 'correct', 'accuracy_percent']].to_string(index=False))
    
    print("\n" + "="*80)
    print("СТАТИСТИКА ПО КЛАСАХ")
    print("="*80)
    
    if 'grade' in results_df.columns:
        grade_stats = results_df.groupby('grade').agg({
            'is_correct': ['count', 'sum']
        }).reset_index()
        grade_stats.columns = ['grade', 'count', 'correct']
        grade_stats['accuracy'] = grade_stats['correct'] / grade_stats['count']
        grade_stats['accuracy_percent'] = grade_stats['accuracy'] * 100
        grade_stats = grade_stats.sort_values('grade')
        
        print("\nКлас | Кількість питань | Правильних | Точність (%)")
        print("-" * 50)
        for _, row in grade_stats.iterrows():
            grade = int(row['grade'])
            count = int(row['count'])
            correct = int(row['correct'])
            accuracy_pct = row['accuracy_percent']
            print(f"{grade:4} | {count:15} | {correct:10} | {accuracy_pct:11.2f}%")
        
        print("\nДетальна таблиця:")
        print(grade_stats[['grade', 'count', 'correct', 'accuracy_percent']].to_string(index=False))
    
    print("\n" + "="*80)
    print("ЗАГАЛЬНА СТАТИСТИКА")
    print("="*80)
    total = len(results_df)
    correct = results_df['is_correct'].sum()
    accuracy_pct = (correct / total * 100) if total > 0 else 0
    print(f"\nВсього питань: {total}")
    print(f"Правильних відповідей: {correct}")
    print(f"Загальна точність: {accuracy_pct:.2f}%")
else:
    print("Помилка: у results_df немає колонки 'subject'")
    print(f"Доступні колонки: {list(results_df.columns)}")

In [ ]:
# Форматування статистики з відсотками
print("="*80)
print("РЕЗУЛЬТАТИ БЕНЧМАРКУ")
print("="*80)

print(f"\n📊 ЗАГАЛЬНА СТАТИСТИКА:")
print(f"   Всього питань: {metrics['total_questions']}")
print(f"   Правильних відповідей: {int(metrics['correct_answers'])}")
print(f"   Загальна точність: {metrics['accuracy_percent']:.2f}%")

print("\n" + "="*80)
print("📚 СТАТИСТИКА ПО ПРЕДМЕТАХ")
print("="*80)

if 'by_subject' in metrics:
    subject_df = metrics['by_subject'].copy()
    subject_df['accuracy_percent'] = subject_df['accuracy'] * 100
    subject_df = subject_df.sort_values('accuracy_percent', ascending=False)
    
    print("\nПредмет              | Питань | Правильних | Точність (%)")
    print("-" * 60)
    for _, row in subject_df.iterrows():
        subject = row['subject']
        count = int(row['count'])
        correct = int(row['correct'])
        accuracy_pct = row['accuracy_percent']
        bar_length = int(accuracy_pct / 2)
        bar = "█" * bar_length + "░" * (50 - bar_length)
        print(f"{subject:20} | {count:6} | {correct:10} | {accuracy_pct:6.2f}% {bar[:50]}")
    
    print("\nДетальна таблиця:")
    display(subject_df[['subject', 'count', 'correct', 'accuracy_percent']].style.format({
        'count': '{:.0f}',
        'correct': '{:.0f}',
        'accuracy_percent': '{:.2f}%'
    }).set_caption("Статистика по предметах"))

print("\n" + "="*80)
print("🎓 СТАТИСТИКА ПО КЛАСАХ")
print("="*80)

if 'by_grade' in metrics:
    grade_df = metrics['by_grade'].copy()
    grade_df['accuracy_percent'] = grade_df['accuracy'] * 100
    grade_df = grade_df.sort_values('grade')
    
    print("\nКлас | Питань | Правильних | Точність (%)")
    print("-" * 50)
    for _, row in grade_df.iterrows():
        grade = int(row['grade'])
        count = int(row['count'])
        correct = int(row['correct'])
        accuracy_pct = row['accuracy_percent']
        bar_length = int(accuracy_pct / 2)
        bar = "█" * bar_length + "░" * (50 - bar_length)
        print(f"  {grade}  | {count:6} | {correct:10} | {accuracy_pct:6.2f}% {bar[:50]}")
    
    print("\nДетальна таблиця:")
    display(grade_df[['grade', 'count', 'correct', 'accuracy_percent']].style.format({
        'grade': '{:.0f}',
        'count': '{:.0f}',
        'correct': '{:.0f}',
        'accuracy_percent': '{:.2f}%'
    }).set_caption("Статистика по класах"))

print("\n" + "="*80)
print("📈 ВИСНОВКИ")
print("="*80)

if 'by_subject' in metrics:
    subject_df = metrics['by_subject'].copy()
    subject_df['accuracy_percent'] = subject_df['accuracy'] * 100
    best_subject = subject_df.loc[subject_df['accuracy_percent'].idxmax()]
    worst_subject = subject_df.loc[subject_df['accuracy_percent'].idxmin()]
    
    print(f"\n✅ Найкращий предмет: {best_subject['subject']} ({best_subject['accuracy_percent']:.2f}%)")
    print(f"❌ Найгірший предмет: {worst_subject['subject']} ({worst_subject['accuracy_percent']:.2f}%)")
    
    if 'by_grade' in metrics:
        grade_df = metrics['by_grade'].copy()
        grade_df['accuracy_percent'] = grade_df['accuracy'] * 100
        best_grade = grade_df.loc[grade_df['accuracy_percent'].idxmax()]
        print(f"\n✅ Найкращий клас: {int(best_grade['grade'])} ({best_grade['accuracy_percent']:.2f}%)")

In [ ]:
results_df.columns

Index(['question_id', 'question_text', 'grade', 'subject', 'correct_indices',
       'predicted_index', 'llm_answer_raw', 'is_correct', 'context_length'],
      dtype='object')

In [ ]:
def calculate_metrics(results_df):
    """Обчислює метрики бенчмарку."""
    total = len(results_df)
    correct = results_df['is_correct'].sum()
    accuracy = correct / total if total > 0 else 0
    
    metrics = {
        'total_questions': total,
        'correct_answers': correct,
        'accuracy': accuracy,
        'accuracy_percent': accuracy * 100
    }
    
    if 'subject' in results_df.columns:
        subject_metrics = results_df.groupby('subject').agg({
            'is_correct': ['count', 'sum']
        }).reset_index()
        subject_metrics.columns = ['subject', 'count', 'correct']
        subject_metrics['accuracy'] = subject_metrics['correct'] / subject_metrics['count']
        metrics['by_subject'] = subject_metrics
    
    if 'grade' in results_df.columns:
        grade_metrics = results_df.groupby('grade').agg({
            'is_correct': ['count', 'sum']
        }).reset_index()
        grade_metrics.columns = ['grade', 'count', 'correct']
        grade_metrics['accuracy'] = grade_metrics['correct'] / grade_metrics['count']
        metrics['by_grade'] = grade_metrics
    
    return metrics

In [ ]:
calculate_metrics(results_df)

{'total_questions': 141,
 'correct_answers': np.int64(60),
 'accuracy': np.float64(0.425531914893617),
 'accuracy_percent': np.float64(42.5531914893617),
 'by_subject':            subject  count  correct  accuracy
 0  Історія України     36       26  0.722222
 1          Алгебра     46       15  0.326087
 2  Українська мова     59       19  0.322034,
 'by_grade':    grade  count  correct  accuracy
 0      8     56       19  0.339286
 1      9     85       41  0.482353}